In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import hvplot.pandas
import panel as pn

pn.extension()

data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/DailySales.csv', index_col=0)
data.index = pd.to_datetime(data.index)
print(data.head())


In [ ]:
def get_line_plot(volume, window_size, date_range):
    if volume == 'high':
        selected = ['A', 'F', 'L']
    elif volume == 'medium':
        selected = ['G', 'H', 'J', 'S', 'W']
    elif volume == 'low':
        selected = ['D', 'E', 'M', 'O', 'P', 'T', 'X']
    else:
        selected = ['B', 'C', 'I', 'K', 'N', 'Q', 'R', 'U', 'V', 'Y']
    df = data[selected] # restrict to selected products
    df = df.rolling(window=window_size).mean() # calculate rolling average
    df = df.loc[pd.to_datetime(date_range[0]) : pd.to_datetime(date_range[1])] # restrict to date range
    return df.hvplot.line(frame_width=600, grid=True)

In [ ]:
time_title = '## Time-Series'

volumes = ['high', 'medium', 'low', 'very low']

volume = pn.widgets.Select(name='Volume', options=volumes, width=200)
day_avg = pn.widgets.IntSlider(name='Day average', value=7, start=1, end=21, width=200)
time_date_range = pn.widgets.DateRangeSlider(name='Dates', width=200,
                                        value=(data.index[0], data.index[-1]),
                                        start=data.index[0], end=data.index[-1])

time_row = pn.Row(
    pn.Column(time_title, volume, day_avg, time_date_range),
    get_line_plot(volume.value, day_avg.value, time_date_range.value)
)

def update_time_row(event):
    time_row[1].object = get_line_plot(volume.value, day_avg.value, time_date_range.value)

volume.param.watch(update_time_row, 'value')
day_avg.param.watch(update_time_row, 'value')
time_date_range.param.watch(update_time_row, 'value')

time_row.servable()